# Where are we winning?
> Inflection-sensitive charts for detecting successful interventions, à la _minutephysics_' "How To Tell If We're Beating COVID-19". States will plummet off the diagonal when they get the virus under control.

- author: Daniel Cox
- categories: [inflection, US, states]
- image: images/where-are-we-winning.png
- permalink: /where-are-we-winning/

In [ ]:
#hide
%matplotlib inline
import sys
import math
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from datetime import datetime
from IPython.display import HTML
# import plotly.io as pio
# # pio.renderers.default = 'notebook_connected'
# pio.renderers.default = 'notebook+'

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
#hide
states_url = "https://covidtracking.com/api/states/daily"
case_threshold = 100 # TODO I don't want to drop states below 100

r = requests.get(states_url)
states_df = pd.DataFrame(r.json())
states_df['date'] = pd.to_datetime(states_df.date, format="%Y%m%d")
states_df = states_df[['date', 'state', 'positive']].sort_values('date')
states_df = states_df.rename(columns={'positive': 'confirmed'})
cols = {}
for state in states_df.state.unique():
    cases = states_df[(states_df.state == state) & (states_df.confirmed > case_threshold)]
    cases = cases.reset_index().confirmed.reset_index(drop=True)
    if len(cases) > 1:
        cols[state] = cases

df = states_df.reset_index()
# df

In [ ]:
# hide
df = (df.assign(daily_new=df.groupby('state', as_index=False)[['confirmed']]
                            .diff().fillna(0)
                            .reset_index(0, drop=True)))

In [ ]:
#hide
df = (df.assign(avg_daily_new=df.groupby('state', as_index=False)[['daily_new']]
                                .rolling(7).mean()
                                .reset_index(0, drop=True)))

In [ ]:
#hide
state_names = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AS": "American Samoa",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "DC": "District Of Columbia",
    "FM": "Federated States Of Micronesia",
    "FL": "Florida",
    "GA": "Georgia",
    "GU": "Guam",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MH": "Marshall Islands",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "MP": "Northern Mariana Islands",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PW": "Palau",
    "PA": "Pennsylvania",
    "PR": "Puerto Rico",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VI": "Virgin Islands",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming"
}

In [ ]:
#hide
df['day'] = df.date.apply(lambda x: x.date()).apply(str)
df = df.sort_values(by='day')
dfc = df[df.avg_daily_new > 0]

In [ ]:
#hide
days = dfc.day.unique().tolist()
states = dfc.state.unique().tolist()
states.sort()

In [ ]:
#hide
# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# fill in most of layout
fig_dict["layout"]["height"] = 700
fig_dict["layout"]["width"] = 900
fig_dict["layout"]["xaxis"] = {"range": [np.log10(5), np.log10(dfc['confirmed'].max() + 5000)], "title": "Total Confirmed Cases (log scale)", "type": "log"}
fig_dict["layout"]["yaxis"] = {"range": [np.log10(1), np.log10(dfc['avg_daily_new'].max() + 500)], "title": "Average Daily New Cases (log scale)", "type": "log"}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["sliders"] = {
    "args": [
        "transition", {
            "duration": 100,
            "easing": "cubic-in-out"
        }
    ],
    "initialValue": min(days),
    "plotlycommand": "animate",
    "values": days,
    "visible": True
}

# buttons
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 300, "redraw": True},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "linear"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.05,
        "xanchor": "right",
        "y": 0.05,
        "yanchor": "top"
    }
]

# sliders
sliders_dict = {
    "active": len(days)-1,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
#         "prefix": "Date: ",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 100},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

In [ ]:
#hide
# make data
day = max(days)
for state in states:
    dataset_by_day = dfc[dfc["day"] <= day]
    dataset_by_day_and_state = dataset_by_day[ dataset_by_day["state"]==state ]
    
    data_dict = {
        "x": list(dataset_by_day_and_state["confirmed"]),
        "y": list(dataset_by_day_and_state["avg_daily_new"]),
        "mode": "lines",
        "text": dataset_by_day_and_state[['confirmed', 'avg_daily_new']],
        "name": state,
        'hoverlabel': {'namelength': 0},
        'hovertemplate': '<b>%{hovertext}</b><br>Confirmed: %{x:,d}<br>Average Daily: %{y:,.2f}',
        'hovertext': dataset_by_day_and_state['state'].apply(lambda s: state_names.get(s, '??') + f' ({s})'),
    }
    fig_dict["data"].append(data_dict)

# make frames
for day in days:
    frame = {"data": [], "name": day}
    for state in states:
        dataset_by_day = dfc[dfc["day"] <= day]
        dataset_by_day_and_state = dataset_by_day[
            dataset_by_day["state"] == state]

        data_dict = {
            "x": list(dataset_by_day_and_state["confirmed"]),
            "y": list(dataset_by_day_and_state["avg_daily_new"]),
            "mode": "lines",
            "text": dataset_by_day_and_state[['confirmed', 'avg_daily_new']],
            "name": state
        }
        frame["data"].append(data_dict)

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [day],
        {"frame": {"duration": 100, "redraw": True},
         "mode": "immediate",
         "transition": {"duration": 100, 'easing': 'linear'}}
    ],
        "label": day,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)

In [ ]:
#hide
fig_dict["layout"]["sliders"] = [sliders_dict]
fig = go.Figure(fig_dict)

In [ ]:
#hide
fig.show()

In [ ]:
#hide_input
HTML(fig.to_html())

> Tip: Click on states in the legend to select/de-select, and double-click to isolate a single state. Hovor over a line to disambiguate, and to see exact numbers. Use the slider or press Play to see the plot at different points in time.

## Explanation

The exponential growth stage of a pandemic must end sometime, either as the virus runs out of people to infect, or as societies get it under control. However, it can be difficult to tell exactly when exponential growth is ending, for several reasons:

* Humans aren't wired to understand exponentials at a glance.
* It can be difficult to compare regions with differing first-infection dates and populations.
* The news tends to report individual data points, without the contextual information necessary to interpret it.

This visualization plots the (sliding average of) daily new cases against the total cases, for each US state (with other countries and regions to come). This has the advantage of aligning all of them onto a baseline trajectory of exponential growth, with a very clear downward plummet when a given state gets the virus under control.

_minutephysics_ has an excellent video on this visualization type, [How To Tell If We're Beating COVID-19](https://youtu.be/54XLXg4fYsc).

## Caveats



This visualization was made by [Daniel Cox](https://twitter.com/danielpcox), with thanks to Henry of _minutephysics_ for [How To Tell If We're Beating COVID-19](https://youtu.be/54XLXg4fYsc).

[^1]:  Data sourced from ["The COVID Tracking Project"](https://covidtracking.com/). Updated hourly by [GitHub Actions](https://github.com/features/actions).